In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [2]:
dataset = Dataset.ninety
current_time = datetime.now().strftime("%d%m%Y-%H%M%S")
extractor_train, extractor_val = prep_data_single(dataset, 8)
train_gen, val_gen = prep_data(dataset, extractor_train, extractor_val, 8, 4)

Found 5400 files belonging to 90 classes.
Using 4590 files for training.
Found 5400 files belonging to 90 classes.
Using 810 files for validation.


In [3]:
inception = InceptionV3(classifier_activation=None)
inception.trainable = True

inputs = keras.Input(shape=(299, 299, 3))
flip_aug = experimental.preprocessing.RandomFlip()(inputs)
rotate_aug = experimental.preprocessing.RandomRotation(0.2)(flip_aug)
preprocessing = preprocess_input(rotate_aug)
extractor = inception(preprocessing, training=False)
inception_model = keras.Model(inputs=inputs, outputs=extractor)
print(inception_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 299, 299, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 299, 299, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 299, 299, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 299, 299, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, 1000)              23851784  
Total params: 23,851,784
Trainable params: 23,817,352
Non-trainable params: 34,432
____________________________________________

In [4]:
extractor_logdir = "logs/time_distributed/{0}_{1}/extractor".format(str(dataset), current_time)
extractor_tensorboard_callback = keras.callbacks.TensorBoard(log_dir=extractor_logdir)

extractor_model_path = "models/time_distributed/{0}_{1}/extractor/savefile.hdf5".format(str(dataset), current_time)
extractor_model_save_callback = keras.callbacks.ModelCheckpoint(filepath=extractor_model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [5]:
feature_extractor = keras.Sequential([inception_model, keras.layers.Dense(train_gen.num_classes(), activation='softmax')])
feature_extractor.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(feature_extractor.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 1000)              23851784  
_________________________________________________________________
dense (Dense)                (None, 291)               291291    
Total params: 24,143,075
Trainable params: 24,108,643
Non-trainable params: 34,432
_________________________________________________________________
None


In [6]:
feature_extractor.fit(extractor_train, validation_data=extractor_val, callbacks=[extractor_tensorboard_callback, extractor_model_save_callback], epochs=15)

Epoch 1/15
6733/6733 [==============================] - 694s 100ms/step - loss: 2.2361 - accuracy: 0.4619 - val_loss: 1.1079 - val_accuracy: 0.6622

Epoch 00001: val_accuracy improved from -inf to 0.66225, saving model to models/time_distributed/Dataset.carabid_25022022-105409/extractor\savefile.hdf5
Epoch 2/15
6733/6733 [==============================] - 681s 101ms/step - loss: 0.8554 - accuracy: 0.7379 - val_loss: 0.8088 - val_accuracy: 0.7544

Epoch 00002: val_accuracy improved from 0.66225 to 0.75442, saving model to models/time_distributed/Dataset.carabid_25022022-105409/extractor\savefile.hdf5
Epoch 3/15
6733/6733 [==============================] - 681s 101ms/step - loss: 0.6161 - accuracy: 0.8060 - val_loss: 0.6207 - val_accuracy: 0.8128

Epoch 00003: val_accuracy improved from 0.75442 to 0.81282, saving model to models/time_distributed/Dataset.carabid_25022022-105409/extractor\savefile.hdf5
Epoch 4/15
6733/6733 [==============================] - 680s 101ms/step - loss: 0.4833 -

In [3]:
feature_extractor = keras.models.load_model("models/time_distributed/NINETY-EXTRACTOR/extractor/savefile.hdf5")
inception_model = feature_extractor.layers[0].layers[-1]
inception_model.trainable = False

classifier_model = keras.Sequential([
    InputLayer(input_shape=(None, 299, 299, 3)),
    TimeDistributed(inception_model),
    Bidirectional(LSTM(1000)),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dense(train_gen.num_classes(), activation='softmax')
])

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1, decay_steps=1000, decay_rate=0.9)
classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(classifier_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 1000)        23851784  
_________________________________________________________________
bidirectional (Bidirectional (None, 2000)              16008000  
_________________________________________________________________
dropout (Dropout)            (None, 2000)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              2001000   
_________________________________________________________________
dense_1 (Dense)              (None, 90)                90090     
Total params: 41,950,874
Trainable params: 18,099,090
Non-trainable params: 23,851,784
_________________________________________________________________
None


In [4]:
logdir = "logs/time_distributed/{0}_{1}/classifier".format(str(dataset), current_time)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "models/time_distributed/{0}_{1}/classifier/savefile.hdf5".format(str(dataset), current_time)
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [5]:
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [6]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
classifier_model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)

Epoch 1/20
139/139 [==============================] - 71s 414ms/step - loss: 4.2230 - accuracy: 0.1070 - val_loss: 3.7876 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to models/time_distributed/Dataset.ninety_27022022-002434/classifier\savefile.hdf5
Epoch 2/20
139/139 [==============================] - 51s 366ms/step - loss: 3.3858 - accuracy: 0.5665 - val_loss: 3.0544 - val_accuracy: 0.8750

Epoch 00002: val_accuracy improved from 0.50000 to 0.87500, saving model to models/time_distributed/Dataset.ninety_27022022-002434/classifier\savefile.hdf5
Epoch 3/20
139/139 [==============================] - 49s 351ms/step - loss: 2.5763 - accuracy: 0.8885 - val_loss: 2.3292 - val_accuracy: 0.9524

Epoch 00003: val_accuracy improved from 0.87500 to 0.95238, saving model to models/time_distributed/Dataset.ninety_27022022-002434/classifier\savefile.hdf5
Epoch 4/20
139/139 [==============================] - 47s 342ms/step - loss: 1.8536 - accuracy: 0